<a href="https://colab.research.google.com/github/sokanaid/CourseProject2022BiologicalMath/blob/main/model_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение моделей предсказания числинности на плато

In [74]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
np.random.seed(0)

In [75]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/sim_tables

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/sim_tables


## Подготовка датасета

In [98]:
file_name = "/content/drive/MyDrive/sim_tables/data_set1.csv"
sim_frame = pd.read_csv(file_name, sep='\t').drop("Unnamed: 0", axis = 1)

In [99]:
sim_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230736 entries, 0 to 230735
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   b              230736 non-null  float64
 1   d              230736 non-null  float64
 2   death_r        230736 non-null  float64
 3   dd             230736 non-null  float64
 4   sd_b           230736 non-null  float64
 5   sd_d           230736 non-null  float64
 6   area_length_x  230736 non-null  float64
 7   initial_pop    230736 non-null  float64
 8   plateau_pop    230736 non-null  float64
dtypes: float64(9)
memory usage: 15.8 MB


In [100]:
print("total count of simulations:", len(sim_frame))
df = sim_frame[sim_frame["area_length_x"]==100.0].drop("area_length_x", axis = 1)
df1 = df.drop_duplicates(subset=['b', 'd', 'death_r', 'dd', 'sd_b', 'sd_d', 'initial_pop']).reset_index(drop=True)
print("total count of simulations to use in learning:", len(df))

total count of simulations: 230736
total count of simulations to use in learning: 162704


In [101]:
# рапределение размеров в изначальном датасете
sim_frame["area_length_x"].value_counts()

100.0     162704
1100.0      2345
3100.0      2345
2100.0      2345
4100.0      2345
2500.0      2343
3300.0      2343
3500.0      2343
2300.0      2343
300.0       2343
1700.0      2343
3900.0      2343
2900.0      2343
4300.0      2343
4500.0      2343
500.0       2343
1900.0      2343
3700.0      2343
5100.0      2343
2700.0      2343
700.0       2343
900.0       2343
4700.0      2343
1300.0      2343
1500.0      2343
5700.0      2342
5500.0      2342
4900.0      2342
5900.0      2342
5300.0      2342
2600.0        14
1600.0        14
3600.0        14
600.0         14
4600.0        13
5600.0        13
Name: area_length_x, dtype: int64

In [102]:
# распределение параметров
for column in list(df.columns.values):
  print(column)
  print(df[column].describe())

b
count    162704.000000
mean          0.668536
std           0.230048
min           0.100000
25%           0.550000
50%           0.750000
75%           0.850000
max           0.950000
Name: b, dtype: float64
d
count    162704.000000
mean          0.359525
std           0.220228
min           0.100000
25%           0.200000
50%           0.300000
75%           0.500000
max           0.900000
Name: d, dtype: float64
death_r
count    162704.000000
mean          4.986707
std           3.064961
min           0.100000
25%           2.000000
50%           5.000000
75%           8.000000
max          10.000000
Name: death_r, dtype: float64
dd
count    162704.000000
mean          0.549765
std           0.287154
min           0.100000
25%           0.300000
50%           0.550000
75%           0.800000
max           1.000000
Name: dd, dtype: float64
sd_b
count    162704.000000
mean          0.725316
std           0.167597
min           0.500000
25%           0.600000
50%           0.750000
75%

In [103]:
X = df.drop("plateau_pop", axis =1)
y = df["plateau_pop"]

In [104]:
X.head()

,b,d,death_r,dd,sd_b,sd_d,initial_pop
0,0.45,0.3,7.0,0.7,0.95,0.80,1.0
1,0.95,0.8,9.0,0.2,0.80,0.95,1.0
2,0.35,0.2,1.0,0.6,0.80,0.65,1.0
3,0.35,0.2,6.0,0.1,0.65,0.65,1.0
4,0.65,0.1,3.0,0.8,0.65,0.50,1.0


In [105]:
y.head()

0    2.403756e-111
1    4.701355e-110
2    8.296506e-112
3    3.113991e-109
4     1.366164e+02
Name: plateau_pop, dtype: float64

In [106]:
X_train, X_t, y_train, y_t = train_test_split(X, y, test_size=0.3)
X_train

,b,d,death_r,dd,sd_b,sd_d,initial_pop
228523,0.35,0.1,5.0,0.3,0.50,0.95,1.0
127757,0.75,0.7,3.0,0.6,0.95,0.65,50.0
11516,0.95,0.1,4.0,0.2,0.95,0.50,50.0
155180,0.95,0.5,8.0,0.7,0.80,0.65,50.0
184417,0.75,0.6,1.0,0.4,0.95,0.80,100.0
...,...,...,...,...,...,...,...
93018,0.45,0.3,7.0,1.0,0.80,0.80,50.0
11415,0.65,0.3,3.0,0.2,0.80,0.50,1.0
106994,0.75,0.7,2.0,0.9,0.65,0.80,100.0
4073,0.45,0.3,8.0,1.0,0.50,0.50,50.0


In [107]:
X_test,X_val, y_test, y_val = train_test_split(X_t, y_t, test_size=0.4)
X_val

,b,d,death_r,dd,sd_b,sd_d,initial_pop
91606,0.95,0.4,4.0,0.8,0.50,0.95,50.0
48882,0.55,0.1,1.0,0.9,0.50,0.95,1.0
88532,0.65,0.5,10.0,0.9,0.65,0.50,50.0
134766,0.35,0.3,8.0,0.2,0.95,0.50,50.0
147567,0.95,0.7,5.0,0.9,0.80,0.50,1.0
...,...,...,...,...,...,...,...
209802,0.85,0.6,1.0,1.0,0.50,0.65,50.0
45412,0.85,0.3,1.0,0.4,0.50,0.95,1.0
134165,0.45,0.1,6.0,0.1,0.95,0.95,50.0
79151,0.85,0.4,2.0,1.0,0.50,0.80,1.0


In [108]:
print("train len", len(y_train))
print("test len ", len(y_t))
print("val len ", len(y_val))


train len 113892
test len  48812
val len  19525


In [109]:
# нормализация данных
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns = X_val.columns)
X_train

,b,d,death_r,dd,sd_b,sd_d,initial_pop
0,-1.384870,-1.177471,0.003606,-0.872450,-1.343888,1.339422,-0.951992
1,0.354035,1.546173,-0.646877,0.171884,1.339171,-0.449944,0.621960
2,1.223487,-1.177471,-0.321636,-1.220561,1.339171,-1.344627,0.621960
3,1.223487,0.638292,0.979331,0.519995,0.444818,-0.449944,0.621960
4,0.354035,1.092232,-1.297361,-0.524338,1.339171,0.444739,2.228033
...,...,...,...,...,...,...,...
113887,-0.950144,-0.269590,0.654089,1.564329,0.444818,0.444739,0.621960
113888,-0.080691,-0.269590,-0.646877,-1.220561,0.444818,-1.344627,-0.951992
113889,0.354035,1.546173,-0.972119,1.216217,-0.449535,0.444739,2.228033
113890,-0.950144,-0.269590,0.979331,1.564329,-1.343888,-1.344627,0.621960


In [112]:
scaler_y = StandardScaler()
scaler_y.fit(y_train.to_frame())
y_train = scaler_y.transform(y_train.to_frame())
y_test = scaler_y.transform(y_test.to_frame())
y_val = scaler_y.transform(y_val.to_frame())
y_train

array([[-0.19254621],
       [-0.19254621],
       [ 0.436509  ],
       ...,
       [-0.19254621],
       [-0.19254621],
       [-0.19254621]])